## Ejercicios pair programming 5 enero: Sklearn

In [2]:
#Importamos todas las librerias a utilizar
import pandas as pd
import seaborn as sns
import numpy as np
import sidetable
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

pd.options.display.max_columns = None

In [20]:
#Abrimos el archivo y lo guardamos el df
df = pd.read_csv("../datos/tiburon7.csv", index_col=0)

In [21]:
df.head()

,case_number,year,type,country,activity,injury,mes,deceso,genero,especies,edad
0,2018.06.25,2018,Boating,usa,Paddling,"No injury to occupant, outrigger canoe and pad...",Jun,N,F,White,57.0
1,2018.06.03.a,2018,Unprovoked,brazil,Swimming,FATAL,Jun,Y,M,Tiger,18.0
2,2018.05.26.b,2018,Unprovoked,usa,Walking,Lower left leg bitten,May,N,M,Bull,15.0
3,2018.05.24,2018,Provoked,australia,Feeding sharks,Minor bite to hand by captive shark. PROVOKED ...,May,N,M,Grey,32.0
4,2018.05.13.a,2018,Invalid,england,Fishing,Injured by teeth of a dead porbeagle shark he ...,May,N,M,Unspecified,21.0


In [22]:
#Comprobamos los nulos que tenemos en el df
df.isnull().sum()

case_number      0
year             0
type             0
country         10
activity        31
injury           8
mes            154
deceso          99
genero          14
especies         0
edad           158
dtype: int64

1.1. Es el momento de eliminar los nulos:

Reemplazad los valores nulos de la columna age por la media de la edad usando el método SimpleImputer.

In [23]:
#iniciamos el metodo indicando que queremos reemplazar los nulos por la media
imputer = SimpleImputer(strategy='mean', missing_values=np.nan)

In [24]:
#Especificamos que la columna a modificar es edad
imputer = imputer.fit(df[['edad']])

In [25]:
#Creamos una nueva columna con los datos nuevos de edad sin nulos
df['edad_simple'] = imputer.transform(df[['edad']])

In [26]:
#Comprobamos que la nueva columna "edad simple" no tiene nulos
df.isnull().sum()

case_number      0
year             0
type             0
country         10
activity        31
injury           8
mes            154
deceso          99
genero          14
especies         0
edad           158
edad_simple      0
dtype: int64

1.2. Reemplazad los valores nulos de la columna sex por la moda, usando el método SimpleImputer.

💡 Pista 💡 La moda en este tipo de aproximación se indica como most_frequent.

In [27]:

imputer = SimpleImputer(strategy='most_frequent', missing_values=np.nan) #indicamos que queremos cambiarlos por la moda de la columna
imputer = imputer.fit(df[['genero']]) #lo aplicamos a la columna genero
df['genero'] = imputer.transform(df[['genero']]) #renombramos la columna genero  con los nuevos datos


In [28]:
df.isnull().sum()

case_number      0
year             0
type             0
country         10
activity        31
injury           8
mes            154
deceso          99
genero           0
especies         0
edad           158
edad_simple      0
dtype: int64

1.3. Reemplazad los valores nulos de la columna type por el valor más frecuente (la moda) con el método SimpleImputer.

In [29]:
#usamos la columna activity ya que nuestra columna type no tiene nulos
imputer = SimpleImputer(strategy='most_frequent', missing_values=np.nan)  #indicamos que queremos hacer el cambio por la moda
imputer = imputer.fit(df[['activity']])  #aplicandolo a la columna activity
df['activity'] = imputer.transform(df[['activity']])  #renombramos la columna con los nuevos datos sin nulos

In [30]:
df.isnull().sum()  #comprobamos que la columna activity ya no tiene nulos

case_number      0
year             0
type             0
country         10
activity         0
injury           8
mes            154
deceso          99
genero           0
especies         0
edad           158
edad_simple      0
dtype: int64

1.4. Utilizad el método KNN Imputer para reemplazar todos los valores nulos de las columnas numéricas.

In [31]:
numericas = df.select_dtypes(include = np.number)  #sacamos las variables numericas

In [35]:
numericas.head()

,year,edad,edad_simple
0,2018,57.0,57.0
1,2018,18.0,18.0
2,2018,15.0,15.0
3,2018,32.0,32.0
4,2018,21.0,21.0


In [32]:
imputerKNN = KNNImputer(n_neighbors=5)  #indicamos que necesitamos que haga el calculo con 5 "vecinos"
imputerKNN.fit(numericas)

KNNImputer()

In [33]:
numericas_knn= imputerKNN.transform(numericas)

In [34]:
df_knn_imputer = pd.DataFrame(numericas_knn, columns = numericas.columns)  #creamos un df nuevo con los datos sin nulos

In [37]:
df_knn_imputer.isnull().sum()

year           0
edad           0
edad_simple    0
dtype: int64

In [38]:
df["edad_KNN"] = df_knn_imputer["edad"] #añadimos a nuestro df original la columna sin nulos del nuevo df

In [39]:
df.head()

,case_number,year,type,country,activity,injury,mes,deceso,genero,especies,edad,edad_simple,edad_KNN
0,2018.06.25,2018,Boating,usa,Paddling,"No injury to occupant, outrigger canoe and pad...",Jun,N,F,White,57.0,57.0,57.0
1,2018.06.03.a,2018,Unprovoked,brazil,Swimming,FATAL,Jun,Y,M,Tiger,18.0,18.0,18.0
2,2018.05.26.b,2018,Unprovoked,usa,Walking,Lower left leg bitten,May,N,M,Bull,15.0,15.0,15.0
3,2018.05.24,2018,Provoked,australia,Feeding sharks,Minor bite to hand by captive shark. PROVOKED ...,May,N,M,Grey,32.0,32.0,32.0
4,2018.05.13.a,2018,Invalid,england,Fishing,Injured by teeth of a dead porbeagle shark he ...,May,N,M,Unspecified,21.0,21.0,21.0


In [40]:
df.isnull().sum()  #comprobamos

case_number      0
year             0
type             0
country         10
activity         0
injury           8
mes            154
deceso          99
genero           0
especies         0
edad           158
edad_simple      0
edad_KNN         0
dtype: int64

1.5. Utilizad el método Iterative Imputer para reemplazar todos los valores nulos de las columnas numéricas.

In [41]:
numericas.head() #volvemos a usar el df numericas ya que solo necesitamos las variables numericas

,year,edad,edad_simple
0,2018,57.0,57.0
1,2018,18.0,18.0
2,2018,15.0,15.0
3,2018,32.0,32.0
4,2018,21.0,21.0


In [42]:
numericas.isnull().sum()  #comprobamos que en edad aun nos quedan nulos

year             0
edad           158
edad_simple      0
dtype: int64

In [43]:
imputer = IterativeImputer(n_nearest_features=None, imputation_order='ascending')  #indicamos que queremos que vaya comparando de las columnas con menos nulos a las que mas tienen
imputer.fit(numericas)
imputer.transform(numericas)
numericas_iterative = pd.DataFrame(imputer.transform(numericas), columns = numericas.columns) #hacemos un df con las columnas modificadas


In [44]:
df["edad_ite"] = numericas_iterative["edad"]  #creamos una columna en el df original sin los nulos de edad

In [45]:
df.head()

,case_number,year,type,country,activity,injury,mes,deceso,genero,especies,edad,edad_simple,edad_KNN,edad_ite
0,2018.06.25,2018,Boating,usa,Paddling,"No injury to occupant, outrigger canoe and pad...",Jun,N,F,White,57.0,57.0,57.0,57.0
1,2018.06.03.a,2018,Unprovoked,brazil,Swimming,FATAL,Jun,Y,M,Tiger,18.0,18.0,18.0,18.0
2,2018.05.26.b,2018,Unprovoked,usa,Walking,Lower left leg bitten,May,N,M,Bull,15.0,15.0,15.0,15.0
3,2018.05.24,2018,Provoked,australia,Feeding sharks,Minor bite to hand by captive shark. PROVOKED ...,May,N,M,Grey,32.0,32.0,32.0,32.0
4,2018.05.13.a,2018,Invalid,england,Fishing,Injured by teeth of a dead porbeagle shark he ...,May,N,M,Unspecified,21.0,21.0,21.0,21.0


In [46]:
df.isnull().sum()  #en el df original hemos creado 3 columnas de edad sin nulos con cada uno de los metodos que hemos utilizado: simple, iterative y KNN

case_number      0
year             0
type             0
country         10
activity         0
injury           8
mes            154
deceso          99
genero           0
especies         0
edad           158
edad_simple      0
edad_KNN         0
edad_ite         0
dtype: int64

Para comparar las diferencias de los valores en las columnas de edad sin nulos (simple, ite y KNN) creamos una funcion para que nos calcule la media de las columnas numericas

In [65]:
def calcular_medias(df):
    for col in df.columns:

        try:
            
            print(f"La media de la columna {col} es " , df[col].mean())
            print("----------")
        except: 
            print(f"La columna {col} es categorica y no podemos calcular la media")

In [66]:
calcular_medias(df)

La columna case_number es categorica y no podemos calcular la media
La media de la columna year es  1992.4246411483255
----------
La columna type es categorica y no podemos calcular la media
La columna country es categorica y no podemos calcular la media
La columna activity es categorica y no podemos calcular la media
La columna injury es categorica y no podemos calcular la media
La columna mes es categorica y no podemos calcular la media
La columna deceso es categorica y no podemos calcular la media
La columna genero es categorica y no podemos calcular la media
La columna especies es categorica y no podemos calcular la media
La media de la columna edad es  27.889035667107002
----------
La media de la columna edad_simple es  27.889035667107002
----------
La media de la columna edad_KNN es  27.880502392344496
----------
La media de la columna edad_ite es  27.889035667107002
----------


El resulado de las medias de las columnas que hemos modificado son practicamente los mismo excepto en "edad_KNN" donde es ligeramente diferente al tener en cuenta los vecinos

In [67]:
df.head()

,case_number,year,type,country,activity,injury,mes,deceso,genero,especies,edad,edad_simple,edad_KNN,edad_ite
0,2018.06.25,2018,Boating,usa,Paddling,"No injury to occupant, outrigger canoe and pad...",Jun,N,F,White,57.0,57.0,57.0,57.0
1,2018.06.03.a,2018,Unprovoked,brazil,Swimming,FATAL,Jun,Y,M,Tiger,18.0,18.0,18.0,18.0
2,2018.05.26.b,2018,Unprovoked,usa,Walking,Lower left leg bitten,May,N,M,Bull,15.0,15.0,15.0,15.0
3,2018.05.24,2018,Provoked,australia,Feeding sharks,Minor bite to hand by captive shark. PROVOKED ...,May,N,M,Grey,32.0,32.0,32.0,32.0
4,2018.05.13.a,2018,Invalid,england,Fishing,Injured by teeth of a dead porbeagle shark he ...,May,N,M,Unspecified,21.0,21.0,21.0,21.0


2. Guardad el csv

In [68]:
df.to_csv("../datos/tiburon8con_nulos.csv")